In [1]:
import requests
from bs4 import BeautifulSoup

### Web scraping the bigfoot website

In [ ]:
/html/body/table/tbody/tr[1]/td[3]/table/tbody/tr/td/p[10]/text()[2]

In [77]:
html = """
<p>
    <span class="field">OBSERVED:</span>
        "Hi, my name is fuckface. I like pizza."
        <br>
        <br>
        "Maybe I should go to the moon?"
        <br>
        <br>
</p>"""
soup = BeautifulSoup(html)
soup.find("p").getText()

'\nOBSERVED:\n        "Hi, my name is fuckface. I like pizza."\n        \n\n        "Maybe I should go to the moon?"\n        \n\n'

In [82]:
URL = 'https://www.bfro.net/GDB/show_report.asp?id=85'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
features = [item.get_text() for item in soup.select("span.field")]
sighting_d = dict()
for blerg in soup.find_all("p"):
    sighting_d[blerg] = blerg.getText()
# for item in features:
#     if item == 'STATE:' or item == 'COUNTY:':
#         sighting_d[item] = soup.find("span", text=item).next_element.next_element.next_element.next_element
#     else:
#         sighting_d[item] = soup.find("span", text=item).next_sibling

In [83]:
sighting_d[]

{<p><span class="field">YEAR:</span> 2000</p>: 'YEAR: 2000',
 <p><span class="field">SEASON:</span> Spring</p>: 'SEASON: Spring',
 <p><span class="field">MONTH:</span> June</p>: 'MONTH: June',
 <p><span class="field">DATE:</span> ?</p>: 'DATE: ?',
 <p><span class="field">STATE:</span> <a href="/GDB/state_listing.asp?state=OR">Oregon</a></p>: 'STATE: Oregon',
 <p><span class="field">COUNTY:</span> <a href="/GDB/show_county_reports.asp?state=OR&amp;county=Clackamas">Clackamas County</a></p>: 'COUNTY: Clackamas County',
 <p><span class="field">LOCATION DETAILS:</span> It occured between two houses on our property, fairly wooded area.</p>: 'LOCATION DETAILS: It occured between two houses on our property, fairly wooded area.',
 <p><span class="field">NEAREST TOWN:</span> Molalla</p>: 'NEAREST TOWN: Molalla',
 <p><span class="field">NEAREST ROAD:</span> Dickey Prairie road</p>: 'NEAREST ROAD: Dickey Prairie road',
 <p><span class="field">OBSERVED:</span> Right off the bat, I'll tell you I'm 

#### Getting valid URLS

The site uses an id for their sightings, and not all ids lead to valid html pages. My first attempt is to brute force look through all id values up to a certain number (in this case, one of the latest sightings has `id=65996`. 

In [5]:
# id_file = open("ids.txt", "a")
# for i in range(100, 110):
#     id_file.write(str(i)+"\n")
# id_file.close()

The following code was used to pull valid ids from the website. The website urls look like:

> https://www.bfro.net/GDB/show_report.asp?id=IDNUMBER

It appears this ranges from 1 to something around 70,000. This will check each possible id and store valid ones in a text file.

In [9]:
# valid_ids = []
# id_file = open("ids.txt", "a")
# # id_file.write("\n")
# for i in range(5000,20000):
#     URL = f"https://www.bfro.net/GDB/show_report.asp?id={i}"
#     page = requests.get(URL)
#     soup = BeautifulSoup(page.content, 'html.parser')
#     if len(soup) == 5:
#         id_file.write(str(i)+"\n")
#         # print(i)
#         valid_ids.append(i)
# id_file.close()

Opening the file and reading in all the ids:

In [19]:
f = open("ids.txt", "r")
ids = f.read().splitlines()
f.close()

In [21]:
ids[:10]

['60', '70', '76', '77', '80', '81', '83', '85', '89', '96']

In [23]:
def get_data_from_id(id):
    """parse the URL and store the data in a dictionary"""
    sighting_d = dict()
    URL = f"https://www.bfro.net/GDB/show_report.asp?id={id}"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    features = [item.get_text() for item in soup.select("span.field")]
    sighting_d = dict()
    for item in features:
        if item == 'STATE:' or item == 'COUNTY:':
            sighting_d[item] = soup.find("span", text=item).next_element.next_element.next_element.next_element
        else:
            sighting_d[item] = soup.find("span", text=item).next_sibling
    return sighting_d

In [38]:
id = ids[7]
print(id)
print(get_data_from_id(id)["OBSERVED:"])

85
 Right off the bat, I'll tell you I'm a 16 year old kid telling absolutely the truth, so god strike me dead. I hope you don't take my report as bogus because of that. Let me explain my situation to you: I live, and have lived, on the Molalla river for most of my life and never thought twice about walking around at night or anywhere period. About 2 months ago, I was alone in my Grandmas house using the computer, when I decided to go home (my house is only about 50 yards away). As I was stepping outside the most hideous feeling of being alone and fear thumped my heart. I kept walking in terror, when all of the sudden about 30 feet off to my right, I heard a loud thumping sound crash out. From there on, being distorted by my act of running faster than I've ever moved before, I could swear I heard the distinctive two-legged footsteps veering toward me from off the trail between houses. I raced in my house, slammed the door, locked it and sat in the living room to calm down from my stran

In [25]:
all_sightings_d = dict()
for id in ids[:10]:
    all_sightings_d[id] = get_data_from_id(id)

In [28]:
all_sightings_d[ids[4]]

{'Submitted  by  witness Paul R. on Saturday,\xa0July\xa029,\xa02000.': '\n',
 'Late Arriving Campers Hear Running, Stomping, Shrieking, Howling': '\n',
 'YEAR:': ' 1983',
 'SEASON:': ' Summer',
 'MONTH:': ' August',
 'DATE:': ' 12',
 'STATE:': 'Oregon',
 'COUNTY:': 'Lane County',
 'LOCATION DETAILS:': ' It was at Kitson Springs, near the Willamette Divide',
 'NEAREST TOWN:': ' Bend, Oregon',
 'NEAREST ROAD:': ' State Hwy 58',
 'OBSERVED:': ' My veterinarian sister, former girlfriend, and myself had been on a road trip and had finished eating dinner at the Sizzler in Bend, when we headed south to find a camping spot for the night. ',
 'OTHER WITNESSES:': ' 3 persons total.  We were tring to get some sleep.',
 'OTHER STORIES:': ' On another occaison I had found and photograhed bigfoot prints on the trail at Coffee Creek in the Trinity Alps.  I also have seen literature regarding Sasquatch at the ranger station at Castle Crags Park in northern California.',
 'TIME AND CONDITIONS:': ' 1:0

In [29]:
ids[4]

'80'

In [28]:
id = 20000
for i in range(2000):
    id += i
    URL = f"https://www.bfro.net/GDB/show_report.asp?id={id}"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    if len(soup) == 5:
        valid_ids.append(id)
    if i % 

In [29]:
len(valid_ids)

62

In [12]:
for i in range(20000,30000):
    if i % 1000 == 0:
        print(i)

20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
